In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import json
import open3d as o3d
# from collections import Counter
from joblib import Parallel, delayed
from scipy import stats
# parameters
params_3d = {
    # for parallel computing
    "thread_number": 32,                                 
    # for 3d line merging
    "parrallel_thresh_3d":np.cos(2.5*np.pi/180), # tune this
    "overlap_thresh_3d": 0.025,                  # tune this
    # for 3d line pruning
    "degree_threshold": 2,                       # tune this
}

In [10]:
### data path
scene_id = "69e5939669"
# scene_id = "689fec23d7"
# scene_id = "c173f62b15"
# scene_id = "55b2bf8036"
line_mesh_folder = f"/data1/home/lucky/ELSED/dataset/data_jhd/{scene_id}/iphone/line_mesh/"
if not os.path.exists(line_mesh_folder):
    os.makedirs(line_mesh_folder)
### load data
scene_data = np.load(f"/data1/home/lucky/ELSED/code_iphone/result/{scene_id}_results_raw.npy", allow_pickle=True).item()
scene_pose = scene_data["scene_pose"]
print(scene_pose)
scene_intrinsic = scene_data["scene_intrinsic"]
obj_id_label_id = scene_data["obj_id_label_id"]
label_2_semantic_dict = scene_data["label_2_semantic_dict"]
scene_line_2d_end_points = scene_data["scene_line_2d_end_points"]
scene_line_2d_semantic_labels = scene_data["scene_line_2d_semantic_labels"]
scene_line_2d_params = scene_data["scene_line_2d_params"]
scene_line_2d_match_idx = scene_data["scene_line_2d_match_idx"]
scene_line_3d_params = scene_data["scene_line_3d_params"]
scene_line_3d_end_points = scene_data["scene_line_3d_end_points"]
scene_line_3d_image_source = scene_data["scene_line_3d_image_source"]
scene_line_3d_semantic_labels = scene_data["scene_line_3d_semantic_labels"]

{'frame_000000': [[-0.7930862521563116, -0.022060346677087537, 0.6087076484818039, 6.105396965609699], [-0.6090840180164977, 0.03744752668856908, -0.7922202873310288, 3.291934091367147], [-0.005317497718955851, -0.9990543571855539, -0.04313447013911448, 1.6131094952525318], [0.0, 0.0, 0.0, 1.0000000596044458]], 'frame_000020': [[-0.7949955634767969, -0.009322774648723461, 0.6065415349302604, 6.106451429779427], [-0.6064488845667865, 0.03548532253137128, -0.7943294113932738, 3.2921888867774234], [-0.014117538420170137, -0.9993259298196905, -0.03386319210393929, 1.6147263056802592], [0.0, 0.0, 0.0, 1.0000000596044458]], 'frame_000040': [[-0.7882379415741583, -0.008672303059772379, 0.6153073391768936, 6.114690518458571], [-0.6152536799477762, 0.030417208665746893, -0.7877412471202073, 3.2945398290343535], [-0.011883954939190191, -0.9994988728228892, -0.02931039788646984, 1.617002004771334], [0.0, 0.0, 0.0, 1.0000000596044458]], 'frame_000060': [[-0.7538244585671235, 0.010860016430252976, 

In [11]:
# find consistent 3d lines
### preprocessing for 3d line merging
# Precompute reshaped parameters
nnode = len(scene_line_3d_params)
pi_list = np.array([scene_line_3d_params[i][0].reshape(1, 3) for i in range(nnode)])
vi_list = np.array([scene_line_3d_params[i][1].reshape(1, 3) for i in range(nnode)])
project_null_list = np.eye(3) - np.einsum('ijk,ijl->ikl', vi_list, vi_list)
print("constructing the consistent graph")
def find_neighbors(i):
    edges_i = []
    edges_j = []
    if i % 1000 == 0:
        print("finding neighbors in progress:", i/nnode*100,"%")
    cur_image_idices = [scene_line_3d_image_source[i]]
    for j in range(i + 1, nnode):
        if scene_line_3d_image_source[j] not in cur_image_idices:
            if abs(np.dot(vi_list[i], vi_list[j].T)) >= params_3d["parrallel_thresh_3d"]:
                if np.linalg.norm(np.dot(project_null_list[i], (pi_list[i] - pi_list[j]).T)) <= params_3d["overlap_thresh_3d"]:
                    edges_i.append(i)
                    edges_j.append(j)
                    cur_image_idices.append(scene_line_3d_image_source[j])
    return edges_i, edges_j
# for i in range(nnode):
#     result = find_neighbors(i)
results = Parallel(n_jobs=params_3d["thread_number"])(delayed(find_neighbors)(i) for i in range(nnode))
edges_i = []
edges_j = []
for edges_i_, edges_j_ in results:
    for i in range(len(edges_i_)):
        i_ = edges_i_[i]
        j_ = edges_j_[i]
    edges_i.extend(edges_i_)
    edges_j.extend(edges_j_)
np.save(f"/data1/home/lucky/ELSED/code_iphone/result/{scene_id}_edges.npy",
        {"edges_i": edges_i, "edges_j": edges_j})

constructing the consistent graph


finding neighbors in progress: 0.0 %
finding neighbors in progress: 17.244352474564582 %
finding neighbors in progress: 34.488704949129165 %
finding neighbors in progress: 51.73305742369374 %
finding neighbors in progress: 68.97740989825833 %
finding neighbors in progress: 86.2217623728229 %


In [12]:
# just for verification
edge_data = np.load(f"./result/{scene_id}_edges.npy", allow_pickle=True).item()
nnode = len(scene_line_3d_params)
pi_list = np.array([scene_line_3d_params[i][0].reshape(1, 3) for i in range(nnode)])
vi_list = np.array([scene_line_3d_params[i][1].reshape(1, 3) for i in range(nnode)])
project_null_list = np.eye(3) - np.einsum('ijk,ijl->ikl', vi_list, vi_list)
edges_i = np.array(edge_data["edges_i"])
edges_j = np.array(edge_data["edges_j"])
for idx in range(len(edges_i)):
    i = edges_i[idx]
    j = edges_j[idx]
    if abs(np.dot(vi_list[i], vi_list[j].T)) < params_3d["parrallel_thresh_3d"]:
        print(abs(np.dot(vi_list[i], vi_list[j].T)))
    if np.linalg.norm(np.dot(project_null_list[i], (pi_list[i] - pi_list[j]).T)) > params_3d["overlap_thresh_3d"]:
        print(np.linalg.norm(np.dot(project_null_list[i], (pi_list[i] - pi_list[j]).T)))

In [13]:
edge_data = np.load(f"./result/{scene_id}_edges.npy", allow_pickle=True).item()
edges_i = np.array(edge_data["edges_i"])
edges_j = np.array(edge_data["edges_j"])
nnode = len(scene_line_3d_params)
#### starting merging
print("# 3d lines before merging",nnode)
mapping = list(range(0,nnode))
merged_scene_line_3d_params=[]
merged_semantic_label_3d =[]
merged_line_3d_image_idx=[]
merged_scene_line_3d_end_points=[]
edges_i_ = np.concatenate((edges_i,np.array(range(0,nnode))))
edges_j_ = np.concatenate((edges_j,np.array(range(0,nnode))))
vertex_concat = np.concatenate((edges_i_,edges_j_))
# if the line is not observed by enough images, we assume it is a background line and discard it
unique_elements, counts = np.unique(vertex_concat, return_counts=True)
vertex_deleted = unique_elements[counts<params_3d["degree_threshold"]+2]
# delete the 3d lines
for ver in vertex_deleted:
    mapping[ver]=None
index_deleted = []
for i in range(0,len(edges_i_)):
    if edges_i_[i] in vertex_deleted or edges_j_[i] in vertex_deleted:
        index_deleted.append(i)
edges_i_=np.delete(edges_i_,index_deleted)
edges_j_=np.delete(edges_j_,index_deleted)
countt = 0
while len(edges_i_)>0:  
    # find the vertex with largest degree and merge all the neighbors
    vertex_concat = np.concatenate((edges_i_,edges_j_))
    mode_result = stats.mode(vertex_concat)
    if mode_result.count<params_3d["degree_threshold"]+2:
        left_vertex = np.unique(vertex_concat)
        for ver in left_vertex:
            mapping[ver]=None
        break
    most_frequent_index = mode_result.mode
    p_3d = scene_line_3d_params[most_frequent_index][0]
    v_3d = scene_line_3d_params[most_frequent_index][1]
    index_1 = np.where(edges_i_==most_frequent_index)
    index_2 = np.where(edges_j_==most_frequent_index)
    # note that the neighbors contain itself 
    neighbors = np.unique(np.concatenate((edges_j_[index_1], edges_i_[index_2]))) 
    # delete vertex nodes
    for neighbor in neighbors:
        mapping[neighbor]=most_frequent_index
        index_1 = np.where(edges_i_==neighbor)
        index_2 = np.where(edges_j_==neighbor)
        index_delete_neighbor = np.unique(np.concatenate((index_1[0], index_2[0])))
        edges_i_=np.delete(edges_i_,index_delete_neighbor)
        edges_j_=np.delete(edges_j_,index_delete_neighbor)
    # update the end points
    end_points = scene_line_3d_end_points[most_frequent_index]
    sig_dim = np.argmax(np.abs(v_3d))
    for neighbor in neighbors:
        end_points_temp = scene_line_3d_end_points[neighbor]
        if end_points_temp[0][sig_dim]<end_points[0][sig_dim]:
            end_points[0] = end_points_temp[0]
        if end_points_temp[1][sig_dim]>end_points[1][sig_dim]:
            end_points[1] = end_points_temp[1]
    # merge 3d lines
    cluster_semantic_labels = np.array(scene_line_3d_semantic_labels[most_frequent_index])
    for neighbor in neighbors:
        cluster_semantic_labels = np.append(cluster_semantic_labels,scene_line_3d_semantic_labels[neighbor])
    unique_cluster_semantic_lables = np.unique(cluster_semantic_labels)
    unique_cluster_semantic_lables = unique_cluster_semantic_lables[unique_cluster_semantic_lables!=0]
    for labels in unique_cluster_semantic_lables:
        merged_scene_line_3d_params.append(scene_line_3d_params[most_frequent_index])
        merged_semantic_label_3d.append(labels)
        merged_line_3d_image_idx.append(scene_line_3d_image_source[most_frequent_index])
        merged_scene_line_3d_end_points.append(end_points)
    ### output the 3d line with multiple semantic labels
    if len(unique_cluster_semantic_lables)>3:
        sample_d = np.linspace(end_points[0][sig_dim], end_points[1][sig_dim], 300)
        point_sets = []
        for d in sample_d:
            point_sets.append(p_3d + (d - p_3d[sig_dim]) / v_3d[sig_dim] * v_3d)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_sets)
        o3d.io.write_point_cloud(line_mesh_folder + f"multiple_semantic_{countt}.ply", pcd)
        countt+=1
        for k in range(0,len(unique_cluster_semantic_lables)):
            print(f"{label_2_semantic_dict[unique_cluster_semantic_lables[k]]},",end="")
print("# 3d lines after merging:",len(merged_scene_line_3d_params))
### calculate projection error after merging 3d lines
print("calculating projection error after merging")
scene_projection_error_r = {}
scene_projection_error_t = {}
for basename in scene_pose.keys():
    projection_error_r = []
    projection_error_t = []
    intrinsic = scene_intrinsic[basename]
    pose_matrix = np.array(scene_pose[basename])
    line_2d_match_idx = np.array(scene_line_2d_match_idx[basename])
    for j in range(0,len(line_2d_match_idx)):
        if line_2d_match_idx[j]==None or mapping[line_2d_match_idx[j]]==None:
            projection_error_r.append(-1)
            projection_error_t.append(-1)
            continue
        mapping_idx = mapping[line_2d_match_idx[j]]
        n_j = scene_line_2d_params[basename][j].reshape(1,3)
        p_3d = scene_line_3d_params[mapping_idx][0].reshape(3,1)
        v_3d = scene_line_3d_params[mapping_idx][1].reshape(3,1)
        n_j_camera = n_j @ intrinsic
        n_j_camera = n_j_camera / np.linalg.norm(n_j_camera)
        n_j_camera = n_j_camera.reshape(3,1)
        error_rot = (pose_matrix[:3, :3] @ n_j_camera).T @ v_3d
        if error_rot>0.1:
            print(basename,error_rot,j,line_2d_match_idx[j],mapping_idx)
        error_trans = (p_3d.T - np.array(pose_matrix)[:3, 3]) @ (pose_matrix[:3, :3] @ n_j_camera)
        projection_error_r.append(np.abs(error_rot))
        projection_error_t.append(np.abs(error_trans))
    scene_projection_error_r[basename] = projection_error_r
    scene_projection_error_t[basename] = projection_error_t

# 3d lines before merging 5799
wall,floor,window,door,# 3d lines after merging: 558
calculating projection error after merging


In [14]:
####################### output 3d line mesh for visualization ######################
# parallel version
def process_label(i, semantic_label):
    if int(semantic_label) == 0:
        return
    index = np.where(merged_semantic_label_3d == semantic_label)
    print("semantic label:" + f"{label_2_semantic_dict[int(semantic_label)]}" + " number of lines:", len(index[0]))
    point_sets = []
    for j in range(len(index[0])):
        end_points = merged_scene_line_3d_end_points[index[0][j]]
        p, v = merged_scene_line_3d_params[index[0][j]]
        sig_dim = np.argmax(abs(v))
        min_d = end_points[0][sig_dim]
        max_d = end_points[1][sig_dim]
        # get 100 points between min_d and max_d with equal interval
        sample_d = np.linspace(min_d, max_d, 300)
        for d in sample_d:
            point_sets.append(p + (d - p[sig_dim]) / v[sig_dim] * v)
    if point_sets:
        point_sets = np.vstack(point_sets)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_sets)
        o3d.io.write_point_cloud(line_mesh_folder + f"{label_2_semantic_dict[int(semantic_label)]}.ply", pcd)
semantic_labels_all = np.unique(merged_semantic_label_3d)
# Parallel(n_jobs=params_3d["thread_number"])(delayed(process_label)(i, semantic_label) for i, semantic_label in enumerate(semantic_labels_all))
Parallel(n_jobs=8)(delayed(process_label)(i, semantic_label) for i, semantic_label in enumerate(semantic_labels_all))


###################### save data for relocalization experiements ###################################
np.save(f"./result/{scene_id}_results_merged.npy", {
    "scene_pose": scene_pose,
    "scene_intrinsic": scene_intrinsic,
    "obj_id_label_id": obj_id_label_id,
    "label_2_semantic_dict": label_2_semantic_dict,
    ###
    "scene_line_2d_semantic_labels": scene_line_2d_semantic_labels,
    "scene_line_2d_params": scene_line_2d_params,
    "scene_line_2d_end_points": scene_line_2d_end_points,
    "scene_projection_error_r": scene_projection_error_r,
    "scene_projection_error_t": scene_projection_error_t,
    ###
    "merged_scene_line_3d_params": merged_scene_line_3d_params,
    "merged_scene_line_3d_semantic_labels": merged_semantic_label_3d,
    "merged_scene_line_3d_end_points": merged_scene_line_3d_end_points,
    ###
    "params_3d": params_3d
})


semantic label:stool number of lines: 83
semantic label:floor number of lines: 37
semantic label:board number of lines: 3
semantic label:table number of lines: 66
semantic label:ceiling number of lines: 38
semantic label:intercom number of lines: 1
semantic label:shelf number of lines: 55
semantic label:wall number of lines: 129
semantic label:ceiling lamp number of lines: 5
semantic label:window number of lines: 27
semantic label:object number of lines: 7
semantic label:box number of lines: 14
semantic label:roller blinds number of lines: 6
semantic label:container number of lines: 6
semantic label:whiteboard number of lines: 13
semantic label:poster number of lines: 1
semantic label:banner number of lines: 10
semantic label:heater number of lines: 11
semantic label:window frame number of lines: 4
semantic label:windowsill number of lines: 2
semantic label:cabinet number of lines: 9
semantic label:door number of lines: 31


In [15]:
### save merged 3d lines
point_sets=[]
for i in range(len(merged_scene_line_3d_params)):
    end_points = merged_scene_line_3d_end_points[i]
    p, v = merged_scene_line_3d_params[i]
    sig_dim = np.argmax(abs(v))
    min_d = end_points[0][sig_dim]
    max_d = end_points[1][sig_dim]
    # get 100 points between min_d and max_d with equal interval
    sample_d = np.linspace(min_d, max_d, 300)
    for d in sample_d:
        point_sets.append(p + (d - p[sig_dim]) / v[sig_dim] * v)
point_sets = np.vstack(point_sets)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_sets)
o3d.io.write_point_cloud(f"{scene_id}_merged_3d_line_mesh.ply", pcd)
print("write mesh successfully")

write mesh successfully
